<a href="https://colab.research.google.com/github/ekaratnida/Data_Streaming_and_Realtime_Analytics/blob/main/Week06/pyspark/Pyspark_Structure_Streaming_kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://mirrors.estointernet.in/apache/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
#!wget https://www.apache.org/dyn/closer.lua/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz
!pip install findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [10]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName("readFromKafka").master("local").getOrCreate()

In [61]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092") \
  .option("subscribe", "my-first-topic") \
  .option("startingOffsets", "earliest") \
  .load()

#ds = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
display(df)
#ds = df.selectExpr("CAST(value AS STRING)")

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [62]:
personStringDF = df.selectExpr("CAST(value AS STRING)")
display(personStringDF)

DataFrame[value: string]

In [63]:
schema1 = StructType([StructField('id', IntegerType(), True), \
      StructField('firstname', StringType(), True), \
      StructField('middlename', StringType(), True), \
      StructField('lastname', StringType(), True), \
      StructField('dob_year', IntegerType(), True), \
      StructField('dob_month', IntegerType(), True),
      StructField('profession', StringType(), True),
      StructField('gender', StringType(), True),
      StructField('salary', IntegerType(), True)])

In [ ]:
personDF = personStringDF.select(from_json(personStringDF.value,schema1)).selectExpr("*")
personDF.printSchema()
personDF.writeStream.format("console").outputMode("append").start().awaitTermination()